# 🎯 LP Analyzer - アフィリエイトLP自動分析ツール

このNotebookを使用することで、**プログラミング知識がなくても**簡単にランディングページ（LP）を分析できます！

## 📋 このツールでできること

- 🎭 **ペルソナ分析**: どんな人がターゲットなのかを分析
- 💎 **USP抽出**: 商品・サービスの独自の強みを発見
- 🎁 **ベネフィット分析**: お客様が得られる価値を整理
- ✍️ **コピー手法分析**: 使われている心理テクニックを解析
- 📝 **記事作成ガイド**: アフィリエイト記事のポイントを提案

## 🚀 使い方（3ステップで完了！）

1. **セットアップ**: 下のセルを実行してツールを準備
2. **URL入力**: 分析したいLPのURLを入力
3. **分析開始**: ボタンを押して結果を確認

---

**⚠️ 重要**: このツールを使用するには**OpenAI APIキー**が必要です。  
まだお持ちでない場合は [OpenAI公式サイト](https://platform.openai.com/) でアカウントを作成してください。

## 🔧 ステップ1: 必要なツールをインストール

**この作業は初回のみ必要です**  
下のセルを実行すると、分析に必要なプログラムが自動的にインストールされます。

⏱️ **所要時間**: 約2-3分（インターネット速度により変動）

In [ ]:
# 📦 必要なライブラリをインストール
print("🔧 ツールのセットアップを開始します...")
print("⏳ しばらくお待ちください（2-3分程度）")

import sys
import subprocess

# 必要なパッケージのリスト
packages = [
    'playwright>=1.40.0',
    'beautifulsoup4>=4.12.0', 
    'lxml>=4.9.0',
    'openai>=1.0.0',
    'pandas>=2.0.0',
    'asyncio-throttle>=1.0.2',
    'aiofiles>=23.0.0',
    'python-dotenv>=1.0.0',
    'markdownify>=0.11.0',
    'tenacity>=8.2.0',
    'ipywidgets>=8.0.0',
    'tqdm>=4.65.0'
]

# パッケージを1つずつインストール
for i, package in enumerate(packages, 1):
    print(f"📦 {i}/{len(packages)}: {package.split('>=')[0]} をインストール中...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"   ✅ 完了")
    except Exception as e:
        print(f"   ⚠️ エラー: {e}")

print("\n🌐 ブラウザエンジンをインストール中...")
try:
    subprocess.check_call([sys.executable, "-m", "playwright", "install", "chromium", "--with-deps"])
    print("✅ ブラウザエンジンのインストール完了")
except Exception as e:
    print(f"⚠️ ブラウザエンジンのインストールでエラー: {e}")
    print("💡 Google Colabをご利用の場合、このエラーは無視して次に進んでください")

print("\n🎉 セットアップ完了！次のセルに進んでください")

## 📚 ステップ2: 分析プログラムを読み込み

**LP分析の核となるプログラムを準備します**  
このセルでは、ランディングページを分析するためのAI機能を読み込みます。

In [ ]:
# 🧠 LP分析に必要な機能を読み込み
print("🧠 AI分析エンジンを準備中...")

import asyncio
import os
import re
import json
import time
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from pathlib import Path

# UI関連
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from tqdm.notebook import tqdm

# Web関連
try:
    from playwright.async_api import async_playwright
    playwright_available = True
except ImportError:
    playwright_available = False
    print("⚠️ Playwrightが利用できません。代替手段を使用します。")

from bs4 import BeautifulSoup
import requests

# AI関連
from openai import AsyncOpenAI
import openai
from tenacity import retry, stop_after_attempt, wait_exponential

print("✅ ライブラリの読み込み完了")

In [ ]:
# 🔍 ウェブページ抽出機能を定義
print("🔍 ウェブページ分析機能を準備中...")

@dataclass
class ExtractedContent:
    """抽出されたLP内容を格納するクラス"""
    url: str
    title: str = ""
    meta_description: str = ""
    headings: Dict[str, List[str]] = None
    main_text: str = ""
    cta_elements: List[str] = None
    
    def __post_init__(self):
        if self.headings is None:
            self.headings = {}
        if self.cta_elements is None:
            self.cta_elements = []

async def extract_content_playwright(url: str) -> ExtractedContent:
    """Playwrightを使用してウェブページの内容を抽出"""
    content = ExtractedContent(url=url)
    
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            
            # ページにアクセス
            await page.goto(url, timeout=30000, wait_until="networkidle")
            await page.wait_for_load_state("domcontentloaded")
            
            # HTMLを取得
            html = await page.content()
            soup = BeautifulSoup(html, 'lxml')
            
            await browser.close()
            
            # 内容を抽出
            content.title = soup.find('title').get_text().strip() if soup.find('title') else ""
            
            meta_desc = soup.find('meta', attrs={'name': 'description'})
            content.meta_description = meta_desc.get('content', '').strip() if meta_desc else ""
            
            # 見出しを抽出
            for level in range(1, 7):
                tag_name = f'h{level}'
                headings = soup.find_all(tag_name)
                if headings:
                    content.headings[tag_name] = [h.get_text().strip() for h in headings]
            
            # メインテキストを抽出
            for tag in soup(['script', 'style', 'nav', 'header', 'footer']):
                tag.decompose()
            
            main_element = soup.find('main') or soup.find('body') or soup
            content.main_text = main_element.get_text(separator=' ', strip=True)[:8000]  # 8000文字制限
            
            # CTA要素を抽出
            cta_buttons = soup.find_all(['button', 'a'], class_=re.compile(r'btn|button|cta', re.I))
            content.cta_elements = [btn.get_text().strip() for btn in cta_buttons if btn.get_text().strip()]
            
    except Exception as e:
        print(f"⚠️ Playwright抽出エラー: {e}")
        # フォールバック: requestsを使用
        content = await extract_content_requests(url)
    
    return content

async def extract_content_requests(url: str) -> ExtractedContent:
    """requestsを使用してウェブページの内容を抽出（フォールバック）"""
    content = ExtractedContent(url=url)
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'lxml')
        
        # 基本情報を抽出
        content.title = soup.find('title').get_text().strip() if soup.find('title') else ""
        
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        content.meta_description = meta_desc.get('content', '').strip() if meta_desc else ""
        
        # 見出しを抽出
        for level in range(1, 7):
            tag_name = f'h{level}'
            headings = soup.find_all(tag_name)
            if headings:
                content.headings[tag_name] = [h.get_text().strip() for h in headings]
        
        # メインテキストを抽出
        for tag in soup(['script', 'style', 'nav', 'header', 'footer']):
            tag.decompose()
        
        content.main_text = soup.get_text(separator=' ', strip=True)[:8000]
        
    except Exception as e:
        print(f"⚠️ 基本抽出エラー: {e}")
        content.title = "抽出エラー"
        content.main_text = "コンテンツの抽出に失敗しました。"
    
    return content

print("✅ ウェブページ分析機能の準備完了")

In [ ]:
# 🤖 AI分析機能を定義
print("🤖 AI分析機能を準備中...")

@dataclass
class AnalysisResult:
    """分析結果を格納するクラス"""
    url: str
    title: str
    persona_analysis: str = ""
    usp_analysis: str = ""
    benefit_analysis: str = ""
    copywriting_analysis: str = ""
    summary: str = ""
    processing_time: float = 0.0
    cost_estimate: float = 0.0

class OpenAIAnalyzer:
    """OpenAI APIを使用したLP分析クラス"""
    
    def __init__(self, api_key: str):
        self.client = AsyncOpenAI(api_key=api_key)
        self.model = "o4-mini"  # OpenAI o1系リーズニングモデル
        
        # 分析用プロンプトテンプレート
        self.prompts = {
            "persona": {
                "system": "あなたはマーケティングの専門家です。ランディングページの内容から、ターゲット顧客のペルソナを分析してください。",
                "template": """以下のランディングページを分析し、想定されるターゲット顧客のペルソナを詳細に分析してください。

【ページ内容】
タイトル: {title}
メタディスクリプション: {meta_description}
見出し: {headings}
本文: {main_text}

以下の観点でペルソナを分析してください：
1. 年齢層・性別
2. 職業・収入レベル
3. ライフスタイル・価値観
4. 抱えている課題・悩み
5. 情報収集行動
6. 購買決定要因

分析結果は具体的で実用的な内容で回答してください。"""
            },
            "usp": {
                "system": "あなたはマーケティング戦略の専門家です。ランディングページからUSP（独自の強み）と競合優位性を抽出してください。",
                "template": """以下のランディングページの内容からUSP（Unique Selling Proposition）と競合優位性を分析してください。

【ページ内容】
タイトル: {title}
メタディスクリプション: {meta_description}
見出し: {headings}
本文: {main_text}

以下の観点で分析してください：
1. 主要なUSP・独自の強み
2. 競合他社との差別化ポイント
3. 顧客に提供する独自の価値
4. 証拠・根拠となる要素
5. 強みを支える具体的な特徴

実用的で説得力のある分析結果を提供してください。"""
            },
            "benefit": {
                "system": "あなたはコピーライティングの専門家です。ランディングページからベネフィットと訴求キーワードを抽出してください。",
                "template": """以下のランディングページの内容からベネフィットと効果的な訴求キーワードを抽出してください。

【ページ内容】
タイトル: {title}
メタディスクリプション: {meta_description}
見出し: {headings}
本文: {main_text}

以下の観点で分析してください：
1. 顧客が得られる具体的なベネフィット
2. 感情的ベネフィット vs 機能的ベネフィット
3. 効果的な訴求キーワード
4. パワーワード・感情を動かす表現
5. 緊急性・希少性を示す要素
6. 信頼性を高める要素

アフィリエイト記事作成に活用できる実用的な分析結果を提供してください。"""
            },
            "copywriting": {
                "system": "あなたはコピーライティングの専門家です。ランディングページで使用されているコピーライティング手法を分析してください。",
                "template": """以下のランディングページの内容で使用されているコピーライティング手法を分析してください。

【ページ内容】
タイトル: {title}
メタディスクリプション: {meta_description}
見出し: {headings}
本文: {main_text}

以下の手法が使用されているかを分析してください：
1. AIDA（注意→関心→欲求→行動）
2. PAS（問題→共感→解決策）
3. BEAF（Benefit→Evidence→Advantage→Feature）
4. 社会的証明（口コミ、推薦など）
5. 権威性の活用
6. 希少性・緊急性の演出
7. ストーリーテリング

具体的にどの部分でどの手法が使われているかを詳しく分析してください。"""
            }
        }
    
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    async def _call_api(self, system_prompt: str, user_prompt: str) -> tuple:
        """OpenAI APIを呼び出し（リトライ付き）"""
        try:
            response = await self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.3,
                max_tokens=4000
            )
            
            content = response.choices[0].message.content
            tokens = response.usage.total_tokens
            cost = (tokens / 1000) * 0.00015  # o4-miniの概算料金
            
            return content, cost
            
        except Exception as e:
            print(f"⚠️ API呼び出しエラー: {e}")
            raise
    
    async def analyze_content(self, content: ExtractedContent) -> AnalysisResult:
        """LP内容を総合分析"""
        start_time = time.time()
        total_cost = 0.0
        
        # データ準備
        headings_text = "\n".join([f"{level}: {', '.join(headings)}" for level, headings in content.headings.items()])
        
        data = {
            "title": content.title,
            "meta_description": content.meta_description,
            "headings": headings_text,
            "main_text": content.main_text[:6000]  # トークン制限対応
        }
        
        result = AnalysisResult(url=content.url, title=content.title)
        
        # 各分析を順次実行
        analyses = [
            ("persona", "🎭 ペルソナ分析中..."),
            ("usp", "💎 USP分析中..."),
            ("benefit", "🎁 ベネフィット分析中..."),
            ("copywriting", "✍️ コピーライティング分析中...")
        ]
        
        for analysis_type, message in analyses:
            print(message)
            
            prompt_config = self.prompts[analysis_type]
            user_prompt = prompt_config["template"].format(**data)
            
            try:
                analysis_result, cost = await self._call_api(
                    prompt_config["system"], 
                    user_prompt
                )
                total_cost += cost
                
                # 結果を格納
                if analysis_type == "persona":
                    result.persona_analysis = analysis_result
                elif analysis_type == "usp":
                    result.usp_analysis = analysis_result
                elif analysis_type == "benefit":
                    result.benefit_analysis = analysis_result
                elif analysis_type == "copywriting":
                    result.copywriting_analysis = analysis_result
                    
            except Exception as e:
                print(f"⚠️ {analysis_type}分析でエラー: {e}")
                setattr(result, f"{analysis_type}_analysis", f"分析エラー: {e}")
        
        # 総合サマリを生成
        print("📝 総合サマリ生成中...")
        try:
            summary_prompt = f"""以下のLP分析結果から、アフィリエイト記事作成に役立つ要点を3-5つのポイントでまとめてください。

【ペルソナ分析】
{result.persona_analysis[:500]}

【USP分析】
{result.usp_analysis[:500]}

【ベネフィット分析】
{result.benefit_analysis[:500]}

【コピーライティング手法】
{result.copywriting_analysis[:500]}

アフィリエイト記事作成時の具体的なポイントを簡潔にまとめてください。"""
            
            summary, summary_cost = await self._call_api(
                "あなたはアフィリエイトマーケティングの専門家です。LP分析結果から記事作成のポイントをまとめてください。",
                summary_prompt
            )
            result.summary = summary
            total_cost += summary_cost
            
        except Exception as e:
            print(f"⚠️ サマリ生成でエラー: {e}")
            result.summary = "サマリの生成に失敗しました。"
        
        result.processing_time = time.time() - start_time
        result.cost_estimate = total_cost
        
        return result

print("✅ AI分析機能の準備完了")

In [ ]:
# 📊 結果表示機能を定義
print("📊 結果表示機能を準備中...")

def create_html_report(result: AnalysisResult) -> str:
    """分析結果をHTML形式で整形"""
    
    html = f"""
    <div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; max-width: 1000px; margin: 0 auto; padding: 20px; background-color: #f8f9fa;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; border-radius: 15px; text-align: center; margin-bottom: 30px;">
            <h1 style="margin: 0; font-size: 2.5em; font-weight: 300;">🎯 LP分析レポート</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;">{result.title}</p>
            <p style="margin: 5px 0 0 0; font-size: 0.9em; opacity: 0.7;">{result.url}</p>
        </div>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 30px;">
            <div style="background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                <h3 style="color: #4CAF50; margin-top: 0;">⏱️ 処理時間</h3>
                <p style="font-size: 1.5em; font-weight: bold; margin: 0;">{result.processing_time:.1f}秒</p>
            </div>
            <div style="background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
                <h3 style="color: #2196F3; margin-top: 0;">💰 推定コスト</h3>
                <p style="font-size: 1.5em; font-weight: bold; margin: 0;">${result.cost_estimate:.4f}</p>
            </div>
        </div>
    """
    
    # 各分析セクションを追加
    sections = [
        ("🎭 ペルソナ分析", result.persona_analysis, "#FF9800"),
        ("💎 USP・競合優位性", result.usp_analysis, "#9C27B0"),
        ("🎁 ベネフィット分析", result.benefit_analysis, "#4CAF50"),
        ("✍️ コピーライティング手法", result.copywriting_analysis, "#2196F3"),
        ("📝 アフィリエイト記事作成ポイント", result.summary, "#F44336")
    ]
    
    for title, content, color in sections:
        html += f"""
        <div style="background: white; padding: 25px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); margin-bottom: 20px;">
            <h2 style="color: {color}; margin-top: 0; border-bottom: 2px solid {color}; padding-bottom: 10px;">{title}</h2>
            <div style="line-height: 1.6; white-space: pre-wrap;">{content}</div>
        </div>
        """
    
    html += """
        <div style="text-align: center; margin-top: 40px; padding: 20px; background: #e8f5e8; border-radius: 10px;">
            <p style="margin: 0; color: #2e7d32; font-weight: bold;">🎉 分析完了！このレポートをアフィリエイト記事作成にご活用ください</p>
        </div>
    </div>
    """
    
    return html

def create_markdown_report(result: AnalysisResult) -> str:
    """分析結果をMarkdown形式で整形"""
    
    markdown = f"""# 🎯 LP分析レポート

**ページタイトル**: {result.title}  
**URL**: {result.url}  
**分析日時**: {datetime.now().strftime('%Y年%m月%d日 %H:%M:%S')}  
**処理時間**: {result.processing_time:.1f}秒  
**推定コスト**: ${result.cost_estimate:.4f}

---

## 🎭 ペルソナ分析

{result.persona_analysis}

---

## 💎 USP・競合優位性

{result.usp_analysis}

---

## 🎁 ベネフィット分析

{result.benefit_analysis}

---

## ✍️ コピーライティング手法

{result.copywriting_analysis}

---

## 📝 アフィリエイト記事作成ポイント

{result.summary}

---

*このレポートは LP Analyzer により自動生成されました*
"""
    
    return markdown

def create_download_link(content: str, filename: str, content_type: str = "text/plain") -> str:
    """ダウンロードリンクを作成"""
    import base64
    
    # コンテンツをBase64エンコード
    b64_content = base64.b64encode(content.encode('utf-8')).decode('utf-8')
    
    # ダウンロードリンクのHTML
    download_html = f"""
    <a href="data:{content_type};base64,{b64_content}" download="{filename}" 
       style="display: inline-block; padding: 10px 20px; background-color: #4CAF50; color: white; 
              text-decoration: none; border-radius: 5px; font-weight: bold; margin: 5px;">
        📁 {filename} をダウンロード
    </a>
    """
    
    return download_html

print("✅ 結果表示機能の準備完了")

## 🎮 ステップ3: LP分析の実行

**いよいよ分析を開始します！**  
下のセルを実行すると、分析用のインターフェースが表示されます。

### 📝 使い方
1. **OpenAI APIキー**を入力（初回のみ）
2. **分析したいLPのURL**を入力（複数可能）
3. **「分析開始」ボタン**をクリック
4. 結果が表示されるまで少々お待ちください

### ⏱️ 所要時間の目安
- **1つのLP**: 約30秒〜1分
- **複数のLP**: LP数 × 約1分

### 💰 コストの目安
- **1つのLP**: 約$0.01〜0.05（1円〜5円程度）

In [ ]:
# 🎮 LP分析インターフェース
print("🎮 LP分析インターフェースを起動中...")

# UI要素を作成
api_key_input = widgets.Password(
    placeholder='sk-...',
    description='OpenAI APIキー:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

url_input = widgets.Textarea(
    placeholder='分析したいLPのURLを入力してください（複数の場合は1行ずつ）\n\n例：\nhttps://example.com/lp1\nhttps://example.com/lp2',
    description='分析URL:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='100%', height='120px')
)

# サンプルURL設定ボタン
sample_button = widgets.Button(
    description='📝 サンプルURLで試す',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

# 分析開始ボタン
analyze_button = widgets.Button(
    description='🚀 分析開始',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# 結果表示エリア
output_area = widgets.Output()

# グローバル変数（結果保存用）
analysis_results = []

def set_sample_urls(b):
    """サンプルURLを設定"""
    sample_urls = """https://www.apple.com/iphone/
https://www.tesla.com/model3
https://www.airbnb.com/"""
    url_input.value = sample_urls

async def start_analysis(b):
    """分析を開始"""
    global analysis_results
    
    with output_area:
        clear_output(wait=True)
        
        # 入力チェック
        if not api_key_input.value.strip():
            print("❌ OpenAI APIキーを入力してください")
            return
            
        if not url_input.value.strip():
            print("❌ 分析したいURLを入力してください")
            return
        
        # URLリストを作成
        urls = [url.strip() for url in url_input.value.strip().split('\n') if url.strip()]
        
        if not urls:
            print("❌ 有効なURLが見つかりません")
            return
        
        print(f"🎯 {len(urls)}件のLPを分析します\n")
        
        # 分析器を初期化
        try:
            analyzer = OpenAIAnalyzer(api_key_input.value.strip())
        except Exception as e:
            print(f"❌ APIキーの設定エラー: {e}")
            return
        
        analysis_results = []
        total_cost = 0.0
        
        # 各URLを順次分析
        for i, url in enumerate(urls, 1):
            print(f"\n{'='*50}")
            print(f"📊 {i}/{len(urls)}: {url}")
            print(f"{'='*50}")
            
            try:
                # ステップ1: コンテンツ抽出
                print("🔍 ウェブページの内容を抽出中...")
                
                if playwright_available:
                    content = await extract_content_playwright(url)
                else:
                    content = await extract_content_requests(url)
                
                if not content.title or content.title == "抽出エラー":
                    print(f"⚠️ {url} のコンテンツ抽出に失敗しました")
                    continue
                
                print(f"✅ 抽出完了: {content.title}")
                
                # ステップ2: AI分析
                print("\n🤖 AIによる詳細分析を開始...")
                result = await analyzer.analyze_content(content)
                
                analysis_results.append(result)
                total_cost += result.cost_estimate
                
                print(f"✅ 分析完了! 処理時間: {result.processing_time:.1f}秒, コスト: ${result.cost_estimate:.4f}")
                
                # ステップ3: 結果表示
                print("\n📊 結果を表示中...")
                html_report = create_html_report(result)
                display(HTML(html_report))
                
                # ダウンロードリンク作成
                markdown_report = create_markdown_report(result)
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"LP分析レポート_{timestamp}_{i}.md"
                
                download_link = create_download_link(
                    markdown_report, 
                    filename, 
                    "text/markdown"
                )
                display(HTML(download_link))
                
            except Exception as e:
                print(f"❌ {url} の分析でエラーが発生: {e}")
                continue
        
        # 総合結果
        print(f"\n\n{'='*60}")
        print(f"🎉 全ての分析が完了しました！")
        print(f"{'='*60}")
        print(f"📊 分析成功: {len(analysis_results)}/{len(urls)}件")
        print(f"💰 総コスト: ${total_cost:.4f}")
        print(f"⏱️ 総処理時間: {sum(r.processing_time for r in analysis_results):.1f}秒")
        
        if len(analysis_results) > 1:
            # 統合レポート作成
            print("\n📋 統合レポートを作成中...")
            
            integrated_markdown = f"""# 🎯 LP分析統合レポート

**分析日時**: {datetime.now().strftime('%Y年%m月%d日 %H:%M:%S')}  
**分析件数**: {len(analysis_results)}件  
**総コスト**: ${total_cost:.4f}  
**総処理時間**: {sum(r.processing_time for r in analysis_results):.1f}秒

---

"""
            
            for i, result in enumerate(analysis_results, 1):
                integrated_markdown += f"""## {i}. {result.title}

**URL**: {result.url}

### 📝 記事作成ポイント
{result.summary}

---

"""
            
            # 統合レポートのダウンロードリンク
            integrated_filename = f"LP分析統合レポート_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
            integrated_download = create_download_link(
                integrated_markdown,
                integrated_filename,
                "text/markdown"
            )
            
            display(HTML(f"<h3>📋 統合レポート</h3>{integrated_download}"))

# イベントハンドラーを設定
sample_button.on_click(set_sample_urls)
analyze_button.on_click(lambda b: asyncio.create_task(start_analysis(b)))

# UIを表示
display(HTML("""
<div style="background: linear-gradient(135deg, #74b9ff 0%, #0984e3 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
    <h2 style="margin: 0; font-size: 1.8em;">🎮 LP分析ツール</h2>
    <p style="margin: 10px 0 0 0; opacity: 0.9;">URLを入力してAIによるLP分析を開始しましょう！</p>
</div>
"""))

print("\n🔑 まず、OpenAI APIキーを入力してください:")
display(api_key_input)

print("\n📝 分析したいLPのURLを入力してください:")
display(url_input)

print("\n🎮 操作ボタン:")
display(widgets.HBox([sample_button, analyze_button]))

print("\n📊 分析結果がここに表示されます:")
display(output_area)

print("\n✅ 準備完了！上記のフォームを使って分析を開始してください")

## 💡 使い方のコツ

### 🎯 **効果的な分析のために**
- **質の高いLP**を選択: コンテンツが充実しているページほど詳細な分析が可能
- **異なる業界**のLPを比較: 業界ごとの特徴や手法の違いを発見
- **競合サイト**を分析: 自分のビジネスの参考に

### 📝 **分析結果の活用方法**
1. **ペルソナ情報**をアフィリエイト記事のターゲット設定に活用
2. **USP・ベネフィット**を記事の売りポイントとして活用
3. **コピーライティング手法**を自分の記事作成の参考に
4. **記事作成ポイント**を具体的なライティングガイドとして活用

### 🛠️ **トラブルシューティング**
- **コンテンツ抽出エラー**: サイトがアクセス制限をかけている可能性があります
- **API制限エラー**: しばらく時間をおいてから再実行してください
- **分析品質**: 日本語サイトの方が詳細な分析が可能です

---

## 🎉 お疲れ様でした！

このツールを使って効率的なLP分析とアフィリエイト記事作成にお役立てください。  
ご質問やご要望がございましたら、[GitHubリポジトリ](https://github.com/Murasan201/affiliate-lp-analyzer)までお気軽にお寄せください。

**Happy Affiliate Marketing! 🚀**